<a href="https://colab.research.google.com/github/Applied-Machine-Learning-2022/final-project-yeg-ua/blob/ellion0/Yolov5_DeepSort_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git  # clone repo
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.12.0+cu113 (CPU)


# Download data

Get test video from repo and extract the first 2 seconds of it 

In [2]:
%cd /content/Yolov5_StrongSORT_OSNet

# get yolov5m model trained on the crowd-human dataset
!wget -nc https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet/releases/download/v.2.0/crowdhuman_yolov5m.pt -O /content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt

# extract 3 seconds worth of video frames of it
!ffmpeg  -i /content/MiniClass.mp4 -c copy out.mp4

/content/Yolov5_StrongSORT_OSNet
File ‘/content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt’ already there; not retrieving.
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame -

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [9]:
!python track.py --yolo-weights /content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt --classes 0 --strong-sort-weights osnet_x0_25_msmt17.pt --source out.mp4 --save-vid

strong_sort/deep/reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '
track: yolo_weights=['/content/Yolov5_StrongSORT_OSNet/yolov5/crowdhuman_yolov5m.pt'], strong_sort_weights=osnet_x0_25_msmt17.pt, config_strongsort=strong_sort/configs/strong_sort.yaml, source=out.mp4, imgsz=[256, 256], conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, show_vid=False, save_txt=False, save_conf=False, save_crop=False, save_vid=True, nosave=False, classes=[0], agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/track, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, hide_class=False, half=False, dnn=False
'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
YOLOv5 🚀 2022-7-29 Python-3.7.13 torch-1.12.0+cu113 CPU

Fusing layers... 
Model summary: 308 layers, 21041679 parameters

# Show results

In [17]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
show_video('out.mp4')